## WHERE TO OPEN A COFFEE SHOP* IN TORONTO

* The report was supposed to be about veggi restaurants but the no conclusive results due to the low number of cases made me to change the kind of business to analyze

### 1. IMPORTING AND INSTALLING LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 1

##### 1.1 Foursquare credentials

In [3]:
# API Foursquare credentials

CLIENT_ID = 'ZPFNBGEUS2VD5AUKJNBOUV1YWMGTWZGANUSB40RRYFP5CFBA' # your Foursquare ID
CLIENT_SECRET = 'ANCGQR3PXYLKFV2VGGL0KEUKEJKNIM2TFTIJLEU31YWQXSE1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### 2. Importing canadian data

In [4]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
import lxml
df = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df.dropna(subset=['Borough'], inplace=True) # To remove the column Borough

df.head()


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### 2.1 Grouping neighbourhoods by Postal Code

In [5]:
df_groups =pd.DataFrame( df.groupby(['Postal Code','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]))
df_groups.reset_index(inplace=True)
df_groups.head(5)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# Let´s check the size of this dataset
df_groups.shape

(103, 3)

##### 2.2 Importing Codes with latitude and longitude

In [7]:
url2 = 'http://cocl.us/Geospatial_data'
df_lat_lon = pd.read_csv(url2)
df_lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### 2.3 Merging the two previous tables

In [8]:
df_neighborhoods_location = pd.merge(df_groups, df_lat_lon, on='Postal Code')
df_neighborhoods_location.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
# Let`s make a couple of sanity checks before of moving on

    df_toronto = df_neighborhoods_location # let`s rename the main dataset to make it simpler
    
    lat_toronto = df_toronto['Latitude'].mean() # let`s calculate the latitude of toronto to check it with Google
    
    lon_toronto = df_toronto['Longitude'].mean()# let`s calculate the longitud of toronto to check it with Google



In [10]:
print(lat_toronto)
print(lon_toronto)

43.70460773398059
-79.39715291165048


### 2. GENERAL OVERVIEW AND FIRST ASSESSMENTS

#### 2.1 General Map of Toronto

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 2.2 Defining the venues we will retrieve setting a radius of 500

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        
        lat = lat_toronto
        lon = lon_toronto
        LIMIT = 100
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


#### 2.3 Retrieving the 100 main venues

In [13]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [14]:
toronto_venues.shape # let´s take a look to the size of this dataset

(1598, 7)

#### 2.4 Let`s the unique different businesses into these 100 main venues

In [15]:
# Now, let's see the different categories included in these venues

toronto_venues['Venue Category'].unique()

array(['Boat or Ferry', 'Park', 'Beach', 'Construction & Landscaping',
       'Baseball Field', 'Asian Restaurant', 'Auto Garage', 'Pizza Place',
       'Trail', 'Smoke Shop', 'Bus Station', 'Restaurant', 'Coffee Shop',
       'Burger Joint', 'Sandwich Place', 'Discount Store', 'Bakery',
       'Bar', 'Grocery Store', 'Bank', 'Mexican Restaurant', 'Pet Store',
       'Convenience Store', 'Pharmacy', 'Flower Shop', 'Gastropub',
       'Intersection', 'Fish & Chips Shop', 'Bike Shop',
       'Sporting Goods Shop', 'Supermarket', 'Smoothie Shop',
       'Shopping Mall', 'Curling Ice', 'Skating Rink', 'Dessert Shop',
       'Brewery', 'Beer Store', 'Furniture / Home Store',
       'Japanese Restaurant', 'Italian Restaurant',
       'Performing Arts Venue', 'Pool', 'Warehouse Store', 'Gym',
       'Afghan Restaurant', 'Indian Restaurant', 'Yoga Studio',
       'Fast Food Restaurant', 'Gas Station', 'Liquor Store',
       'Turkish Restaurant', 'Fried Chicken Joint', 'Housing Development',
  

#### 2.5 Is there any coffee shop?

In [16]:
# We can check that in the line 26 we have the option I am interested in : 'Coffee Shop' but we can check it by this line of code:

"Coffee Shop" in toronto_venues['Venue Category'].unique()


True

#### 2.6 Let's define a matrix "Neighborhoods x Business" counting the number of businesses

In [17]:
# Now let`s define a table setting the businesses as columns and neighbourhoods as rows

restaurants_toronto = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

restaurants_toronto['Neighborhoods'] = toronto_venues['Neighborhood'] 

columns = [restaurants_toronto.columns[-1]] + list(restaurants_toronto.columns[:-1])
restaurants_toronto = restaurants_toronto[columns]

print(restaurants_toronto.shape)
restaurants_toronto.head(5)

(1598, 131)


,Neighborhoods,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Baby Store,Bagel Shop,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Let`s see the number ( calculated by the mean in order to no duplicate results when the venue is duplicated) of businesses by Neighborhood

hoods_grouped = restaurants_toronto.groupby(["Neighborhoods"]).mean().reset_index()

print(hoods_grouped.shape)
hoods_grouped

(94, 131)


,Neighborhoods,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Baby Store,Bagel Shop,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.200000,0.00,0.0,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
3,Bayview Village,0.000000,0.034483,0.000000,0.000000,0.000000,0.00,0.0,0.00000,0.000000,...,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00000,0.000000,...,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
5,Berczy Park,0.000000,0.024390,0.000000,0.000000,0.048780,0.00,0.0,0.02439,0.024390,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04878,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.100000,0.00,0.0,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00000,0.058824,...,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.013333,0.000000,0.00,0.0,0.00000,0.000000,...,0.013333,0.000000,0.000000,0.013333,0.000000,0.000000,0.000000,0.00000,0.000000,0.013333
9,Caledonia-Fairbanks,0.000000,0.000000,0.066667,0.000000,0.000000,0.00,0.0,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000


#### 2.7 Number of neighborhoods with at least a Coffee Shop

In [19]:
len(hoods_grouped[hoods_grouped["Coffee Shop"] > 0])

51

In [20]:
# Now let`s put the focus only on the Coffee Shops

coffee_frequency = hoods_grouped[["Neighborhoods","Coffee Shop"]]
coffee_frequency2 =  coffee_frequency[coffee_frequency["Coffee Shop"] > 0]
coffee_frequency2.head(20)

,Neighborhoods,Coffee Shop
3,Bayview Village,0.068966
5,Berczy Park,0.073171
6,"Birch Cliff, Cliffside West",0.100000
7,"Brockton, Parkdale Village, Exhibition Place",0.058824
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667
9,Caledonia-Fairbanks,0.133333
12,Central Bay Street,0.062500
14,Church and Wellesley,0.047619
17,"Commerce Court, Victoria Hotel",0.075000
18,Davisville,0.054054


### 3. CLUSTERING

#### 3.1 Let´s define 3 clusters

In [21]:
from sklearn.cluster import KMeans
toclusters = 3

clusters_kmeans = coffee_frequency2.drop(["Neighborhoods"], 1)

#### 3.2 Let´s apply KMeans to do the clustering and merge it with the number of coffee shops

In [22]:
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(clusters_kmeans)

kmeans.labels_[0:20]

array([0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 1, 2],
      dtype=int32)

In [38]:
to_merged = coffee_frequency2.copy()

to_merged["Cluster Labels"] = kmeans.labels_

to_merged.head(100)

,Neighborhoods,Coffee Shop,Cluster Labels
3,Bayview Village,0.068966,0
5,Berczy Park,0.073171,0
6,"Birch Cliff, Cliffside West",0.100000,2
7,"Brockton, Parkdale Village, Exhibition Place",0.058824,0
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667,2
9,Caledonia-Fairbanks,0.133333,2
12,Central Bay Street,0.062500,0
14,Church and Wellesley,0.047619,0
17,"Commerce Court, Victoria Hotel",0.075000,0
18,Davisville,0.054054,0


#### 3.3 Let´s merge the table I have just created with the information from API Foursquare

In [25]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
final_table = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")
print(final_table.shape)
final_table

(1372, 9)


,Neighborhood,Coffee Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,Bayview Village,0.068966,0,43.704608,-79.385975,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
3,Bayview Village,0.068966,0,43.704608,-79.385975,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café
3,Bayview Village,0.068966,0,43.704608,-79.385975,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
3,Bayview Village,0.068966,0,43.704608,-79.385975,XO Gelato,43.705177,-79.388793,Dessert Shop
3,Bayview Village,0.068966,0,43.704608,-79.385975,Zee Grill,43.704985,-79.388476,Seafood Restaurant
3,Bayview Village,0.068966,0,43.704608,-79.385975,Positano,43.704558,-79.388639,Italian Restaurant
3,Bayview Village,0.068966,0,43.704608,-79.385975,Viva Napoli,43.705752,-79.389125,Pizza Place
3,Bayview Village,0.068966,0,43.704608,-79.385975,Sakae Sushi,43.704944,-79.388704,Sushi Restaurant
3,Bayview Village,0.068966,0,43.704608,-79.385975,Florentia Ristorante,43.703594,-79.387985,Italian Restaurant
3,Bayview Village,0.068966,0,43.704608,-79.385975,Starbucks,43.705923,-79.389548,Coffee Shop


#### 3.4 Let´s define the 3 clusters

In [33]:
#Cluster 0
final_table.loc[(final_table['Cluster Labels'] ==0) & (final_table['Venue Category'] == 'Coffee Shop') ]

,Neighborhood,Coffee Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,Bayview Village,0.068966,0,43.704608,-79.385975,Starbucks,43.705923,-79.389548,Coffee Shop
3,Bayview Village,0.068966,0,43.704608,-79.385975,Second Cup,43.704344,-79.388659,Coffee Shop
5,Berczy Park,0.073171,0,43.704608,-79.373306,Starbucks,43.705190,-79.374760,Coffee Shop
5,Berczy Park,0.073171,0,43.704608,-79.373306,Second Cup,43.706093,-79.375216,Coffee Shop
5,Berczy Park,0.073171,0,43.704608,-79.373306,Creeds Coffee Bar,43.706094,-79.375255,Coffee Shop
7,"Brockton, Parkdale Village, Exhibition Place",0.058824,0,43.704608,-79.428191,Starbucks,43.700598,-79.427433,Coffee Shop
12,Central Bay Street,0.062500,0,43.704608,-79.387383,Starbucks,43.705923,-79.389548,Coffee Shop
12,Central Bay Street,0.062500,0,43.704608,-79.387383,Second Cup,43.704344,-79.388659,Coffee Shop
14,Church and Wellesley,0.047619,0,43.704608,-79.383160,Second Cup,43.704344,-79.388659,Coffee Shop
17,"Commerce Court, Victoria Hotel",0.075000,0,43.704608,-79.379817,Starbucks,43.705190,-79.374760,Coffee Shop


In [34]:
#Cluster 1
final_table.loc[(final_table['Cluster Labels'] ==1) & (final_table['Venue Category'] == 'Coffee Shop') ]

,Neighborhood,Coffee Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
33,"Harbourfront East, Union Station, Toronto Islands",0.5,1,43.704608,-79.381752,Starbucks,43.707485,-79.381479,Coffee Shop
80,"Toronto Dominion Centre, Design Exchange",0.5,1,43.704608,-79.381576,Starbucks,43.707485,-79.381479,Coffee Shop


In [35]:
#Cluster 2
final_table.loc[(final_table['Cluster Labels'] ==2) & (final_table['Venue Category'] == 'Coffee Shop') ]

,Neighborhood,Coffee Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,"Birch Cliff, Cliffside West",0.100000,2,43.704608,-79.264848,Tim Hortons,43.706152,-79.268243,Coffee Shop
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667,2,43.704608,-79.394420,Starbucks,43.706859,-79.396950,Coffee Shop
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667,2,43.704608,-79.394420,Starbucks,43.705923,-79.389548,Coffee Shop
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667,2,43.704608,-79.394420,Starbucks,43.705563,-79.397630,Coffee Shop
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667,2,43.704608,-79.394420,Aroma Espresso Bar,43.706944,-79.398413,Coffee Shop
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667,2,43.704608,-79.394420,Starbucks,43.707747,-79.398556,Coffee Shop
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667,2,43.704608,-79.394420,Tim Hortons,43.707501,-79.396619,Coffee Shop
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667,2,43.704608,-79.394420,Second Cup,43.704344,-79.388659,Coffee Shop
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.106667,2,43.704608,-79.394420,Timothy's World Coffee,43.706027,-79.389369,Coffee Shop
9,Caledonia-Fairbanks,0.133333,2,43.704608,-79.453512,Country Style,43.702225,-79.452641,Coffee Shop


#### 3.4 Conclusion

In [37]:
# The best Cluster where to open is the number 1 due to there are only two Starbucks, in the Harbourfront East and Toronto Dominion Centre.

# The following area would be the first option to start the searching.

cluster1 = final_table.loc[(final_table['Cluster Labels'] ==1) & (final_table['Venue Category'] == 'Coffee Shop') ]

map_clusters1 = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=20)

markers_colors={}
markers_colors[1] = 'black'

for lat, lon, cluster in zip(cluster1['Neighborhood Latitude'], cluster1['Neighborhood Longitude'], cluster1['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=1).add_to(map_clusters1)
       
map_clusters1